# **nltk**

In [28]:
import docx
from PyPDF2 import PdfReader
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from transformers import AutoTokenizer
import json
import os

# Загрузка необходимых ресурсов NLTK
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')

def read_text_from_file(file_path):
    """
    Чтение текста из файла (DOCX, PDF, TXT).
    """
    try:
        if file_path.endswith('.docx'):
            doc = docx.Document(file_path)
            text = '\n'.join([paragraph.text for paragraph in doc.paragraphs])
        elif file_path.endswith('.pdf'):
            reader = PdfReader(file_path)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + " "
        else:
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
        return text
    except Exception as e:
        print(f"Ошибка чтения файла: {e}")
        return None

def preprocess_text(text, language='russian'):
    """
    Предобработка текста: токенизация, удаление стоп-слов и пунктуации.
    
    :param text: Исходный текст
    :param language: Язык текста для определения стоп-слов
    :return: Список слов после обработки
    """
    # Приведение к нижнему регистру
    text = text.lower()
    
    # Удаление чисел, специальных символов и знаков пунктуации
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    
    # Токенизация
    tokens = word_tokenize(text)
    
    # Удаление стоп-слов
    stop_words = set(stopwords.words(language))
    filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    
    return filtered_tokens

def create_technical_categories():
    """
    Создание словаря категорий с соответствующими ключевыми словами.
    Эти категории и ключевые слова можно настроить под ваши нужды.
    
    :return: Словарь категорий и связанных ключевых слов
    """
    categories = {
        "Электрические автоматы и выключатели": [
            "автоматический выключатель", "расцепитель", "автомат", "выключатель", "прерыватель", 
            "masterpact", "compact", "контактор", "выключение", "защита", "перегрузка", 
            "короткое замыкание", "отключение", "автоматика", "электрозащита"
        ],
        
        "Электропитание и распределение": [
            "подстанция", "трансформатор", "распределение", "питание", "напряжение", "ток", 
            "сеть", "электроэнергия", "фаза", "нейтраль", "шина", "щит", "панель", "ввод", 
            "рубильник", "электроснабжение", "мощность", "источник питания"
        ],
        
        "Техническое обслуживание оборудования": [
            "обслуживание", "ремонт", "диагностика", "профилактика", "осмотр", "калибровка", 
            "испытание", "тестирование", "смазка", "очистка", "регулировка", "техобслуживание", 
            "запчасть", "износ", "надежность", "срок службы", "эксплуатация"
        ],
        
        "Управление и автоматизация": [
            "контроллер", "система", "автоматизация", "управление", "монитор", "интерфейс", 
            "дисплей", "команда", "параметр", "настройка", "программа", "алгоритм", "режим", 
            "scada", "датчик", "привод", "исполнительный механизм"
        ],
        
        "Инструкция по установке": [
            "установка", "монтаж", "крепление", "соединение", "подключение", "схема", "провод", 
            "кабель", "клемма", "зажим", "инструкция", "руководство", "шаг", "последовательность",
            "инсталляция", "сборка", "демонтаж"
        ],
        
        "Безопасность и защита": [
            "безопасность", "защита", "опасность", "риск", "предупреждение", "авария", "травма", 
            "заземление", "изоляция", "экранирование", "предохранитель", "сигнализация", "тревога", 
            "оповещение", "эвакуация", "спасательный", "предупреждающий"
        ],
        
        "Сертификация и стандарты": [
            "сертификат", "стандарт", "норма", "требование", "соответствие", "гост", "регламент", 
            "директива", "iec", "ieee", "iso", "спецификация", "тестирование", "аттестация", 
            "аккредитация", "качество", "класс"
        ],
    }
    
    return categories

def classify_with_keywords(text, categories_dict, top_n=3):
    """
    Классификация текста на основе ключевых слов и фраз.
    
    :param text: Текст для классификации
    :param categories_dict: Словарь категорий и ключевых слов
    :param top_n: Количество лучших категорий для вывода
    :return: Список кортежей (категория, количество совпадений, относительный вес)
    """
    # Проверка на пустой текст
    if not text:
        return []
    
    # Предобработка текста
    text_lower = text.lower()
    processed_tokens = preprocess_text(text)
    
    # Подсчет встречаемости ключевых слов для каждой категории
    results = {}
    for category, keywords in categories_dict.items():
        score = 0
        keyword_counts = {}
        
        # Проверка вхождений каждого ключевого слова/фразы
        for keyword in keywords:
            # Подсчет точных фраз
            if len(keyword.split()) > 1:
                count = text_lower.count(keyword)
                if count > 0:
                    keyword_counts[keyword] = count
                    score += count * 2  # Более высокий вес для фраз
            # Подсчет отдельных слов
            else:
                # Проверяем наличие в обработанных токенах
                count = processed_tokens.count(keyword)
                if count > 0:
                    keyword_counts[keyword] = count
                    score += count
        
        if score > 0:
            results[category] = {
                'score': score,
                'keywords': keyword_counts
            }
    
    # Расчет относительных весов
    total_score = sum(item['score'] for item in results.values())
    
    if total_score == 0:
        return []
    
    # Формирование отсортированных результатов
    results_list = [(category, data['score'], data['score']/total_score, data['keywords']) 
                    for category, data in results.items()]
    results_list.sort(key=lambda x: x[1], reverse=True)
    
    return results_list[:top_n]

def generate_report(file_path, results, output_format='text'):
    """
    Генерация отчета о классификации.
    
    :param file_path: Путь к классифицируемому файлу
    :param results: Результаты классификации
    :param output_format: Формат вывода (text или json)
    :return: Отчет в указанном формате
    """
    if not results:
        return "Классификация не удалась или не найдено совпадений с категориями."
    
    if output_format == 'json':
        report = {
            "file": os.path.basename(file_path),
            "classification_results": [
                {
                    "category": category,
                    "score": score,
                    "confidence": confidence,
                    "key_terms": list(keywords.keys())
                } for category, score, confidence, keywords in results
            ]
        }
        return json.dumps(report, ensure_ascii=False, indent=2)
    else:
        lines = [f"Результаты классификации для файла: {os.path.basename(file_path)}"]
        lines.append("=" * 80)
        
        for i, (category, score, confidence, keywords) in enumerate(results):
            lines.append(f"{i+1}. {category}")
            lines.append(f"   Уверенность: {confidence:.2%}")
            lines.append(f"   Количество совпадений: {score}")
            
            if keywords:
                lines.append("   Ключевые термины:")
                sorted_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)
                for term, count in sorted_keywords[:5]:  # Показываем топ-5 ключевых термина
                    lines.append(f"     - {term}: {count}")
            
            lines.append("-" * 40)
        
        return "\n".join(lines)

def classify_technical_document(file_path, custom_categories=None):
    """
    Полный процесс классификации технического документа.
    
    :param file_path: Путь к файлу
    :param custom_categories: Пользовательские категории или None для использования предопределенных
    :return: Результаты классификации
    """
    # Чтение текста из файла
    text = read_text_from_file(file_path)
    if not text:
        print(f"Не удалось прочитать файл: {file_path}")
        return None
    
    # Получение категорий
    if custom_categories is None:
        categories = create_technical_categories()
    else:
        categories = custom_categories
    
    # Классификация
    results = classify_with_keywords(text, categories)
    
    # Генерация и вывод отчета
    report = generate_report(file_path, results)
    print(report)
    
    return results

# Пример использования
if __name__ == "__main__":
    file_path = 'Masterpact.pdf'  # Замените на путь к вашему файлу
    results = classify_technical_document(file_path)
    
    # Если вы хотите сохранить отчет в JSON
    # json_report = generate_report(file_path, results, output_format='json')
    # with open('classification_report.json', 'w', encoding='utf-8') as f:
    #     f.write(json_report)

Результаты классификации для файла: Masterpact.pdf
1. Электрические автоматы и выключатели
   Уверенность: 55.90%
   Количество совпадений: 161
   Ключевые термины:
     - masterpact: 70
     - выключатель: 57
     - расцепитель: 17
     - отключение: 12
     - автоматический выключатель: 1
----------------------------------------
2. Электропитание и распределение
   Уверенность: 15.62%
   Количество совпадений: 45
   Ключевые термины:
     - напряжение: 13
     - ток: 13
     - питание: 10
     - панель: 5
     - нейтраль: 1
----------------------------------------
3. Техническое обслуживание оборудования
   Уверенность: 15.62%
   Количество совпадений: 45
   Ключевые термины:
     - испытание: 19
     - эксплуатация: 7
     - тестирование: 5
     - осмотр: 4
     - надежность: 3
----------------------------------------


# transformers AutoTokenizer

In [8]:
import docx
from PyPDF2 import PdfReader
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from transformers import AutoTokenizer
from pymorphy2 import MorphAnalyzer
import json
import os

import pymorphy2
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures


# Загрузка необходимых ресурсов NLTK
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')

def read_text_from_file(file_path):
    """
    Чтение текста из файла (DOCX, PDF, TXT).
    """
    try:
        if file_path.endswith('.docx'):
            doc = docx.Document(file_path)
            text = '\n'.join([paragraph.text for paragraph in doc.paragraphs])
        elif file_path.endswith('.pdf'):
            reader = PdfReader(file_path)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + " "
        else:
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
        return text
    except Exception as e:
        print(f"Ошибка чтения файла: {e}")
        return None


def preprocess_text(text, language='russian'):
    """
    Предобработка текста с сохранением словосочетаний и лемматизацией.
    
    :param text: Исходный текст
    :param language: Язык текста для определения стоп-слов
    :return: Список слов и словосочетаний после обработки
    """
    # Приведение к нижнему регистру
    text = text.lower()
    
    # Удаление чисел, специальных символов и знаков пунктуации
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    
    # Базовая токенизация
    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
    tokens = tokenizer.tokenize(text)
    
    # Удаление стоп-слов
    stop_words = set(stopwords.words(language))
    filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    
    # Лемматизация отдельных слов
    morph = pymorphy2.MorphAnalyzer()
    lemmatized_tokens = [morph.parse(token)[0].normal_form for token in filtered_tokens]
    
    # Находим биграммы и триграммы (словосочетания из 2-3 слов)
    finder_bigrams = BigramCollocationFinder.from_words(tokens)
    finder_trigrams = TrigramCollocationFinder.from_words(tokens)
    
    # Отфильтруем стоп-слова из словосочетаний
    finder_bigrams.apply_word_filter(lambda w: w in stop_words or len(w) <= 2)
    finder_trigrams.apply_word_filter(lambda w: w in stop_words or len(w) <= 2)
    
    # Находим словосочетания с высокой статистической связью
    bigrams = finder_bigrams.nbest(BigramAssocMeasures.pmi, 50)  # top 50 биграмм
    trigrams = finder_trigrams.nbest(TrigramAssocMeasures.pmi, 30)  # top 30 триграмм
    
    # Лемматизируем словосочетания
    lemmatized_bigrams = []
    for w1, w2 in bigrams:
        lemma1 = morph.parse(w1)[0].normal_form
        lemma2 = morph.parse(w2)[0].normal_form
        lemmatized_bigrams.append(f"{lemma1}_{lemma2}")
    
    lemmatized_trigrams = []
    for w1, w2, w3 in trigrams:
        lemma1 = morph.parse(w1)[0].normal_form
        lemma2 = morph.parse(w2)[0].normal_form
        lemma3 = morph.parse(w3)[0].normal_form
        lemmatized_trigrams.append(f"{lemma1}_{lemma2}_{lemma3}")
    
    # Добавим словосочетания к отдельным словам
    result_tokens = lemmatized_tokens + lemmatized_bigrams + lemmatized_trigrams
    
    return result_tokens

def create_technical_categories():
    """
    Создание словаря категорий с соответствующими ключевыми словами.
    Эти категории и ключевые слова можно настроить под ваши нужды.
    
    :return: Словарь категорий и связанных ключевых слов
    """
    categories = {
        "Электрические автоматы и выключатели": [
            "автоматический выключатель", "расцепитель", "автомат", "выключатель", "прерыватель", 
            "masterpact", "compact", "контактор", "выключение", "защита", "перегрузка", 
            "короткое замыкание", "отключение", "автоматика", "электрозащита"
        ],
        
        "Электропитание и распределение": [
            "подстанция", "трансформатор", "распределение", "питание", "напряжение", "ток", 
            "сеть", "электроэнергия", "фаза", "нейтраль", "шина", "щит", "панель", "ввод", 
            "рубильник", "электроснабжение", "мощность", "источник питания"
        ],
        
        "Техническое обслуживание оборудования": [
            "обслуживание", "ремонт", "диагностика", "профилактика", "осмотр", "калибровка", 
            "испытание", "тестирование", "смазка", "очистка", "регулировка", "техобслуживание", 
            "запчасть", "износ", "надежность", "срок службы", "эксплуатация"
        ],
        
        "Управление и автоматизация": [
            "контроллер", "система", "автоматизация", "управление", "монитор", "интерфейс", 
            "дисплей", "команда", "параметр", "настройка", "программа", "алгоритм", "режим", 
            "scada", "датчик", "привод", "исполнительный механизм"
        ],
        
        "Инструкция по установке": [
            "установка", "монтаж", "крепление", "соединение", "подключение", "схема", "провод", 
            "кабель", "клемма", "зажим", "инструкция", "руководство", "шаг", "последовательность",
            "инсталляция", "сборка", "демонтаж"
        ],
        
        "Безопасность и защита": [
            "безопасность", "защита", "опасность", "риск", "предупреждение", "авария", "травма", 
            "заземление", "изоляция", "экранирование", "предохранитель", "сигнализация", "тревога", 
            "оповещение", "эвакуация", "спасательный", "предупреждающий"
        ],
        
        "Сертификация и стандарты": [
            "сертификат", "стандарт", "норма", "требование", "соответствие", "гост", "регламент", 
            "директива", "iec", "ieee", "iso", "спецификация", "тестирование", "аттестация", 
            "аккредитация", "качество", "класс"
        ],
    }
    
    return categories

def classify_with_keywords(text, categories_dict, top_n=3):
    """
    Классификация текста на основе ключевых слов и фраз с учетом словосочетаний.
    
    :param text: Текст для классификации
    :param categories_dict: Словарь категорий и ключевых слов
    :param top_n: Количество лучших категорий для вывода
    :return: Список кортежей (категория, количество совпадений, относительный вес)
    """
    if not text:
        return []
    
    # Предобработка текста
    text_lower = text.lower()
    
    # Сначала найдем все точные фразы до токенизации
    results = {}
    for category, keywords in categories_dict.items():
        score = 0
        keyword_counts = {}
        
        # Проверка вхождений каждого ключевого словосочетания
        for keyword in keywords:
            # Подсчет словосочетаний
            if len(keyword.split()) > 1:
                count = text_lower.count(keyword)
                if count > 0:
                    keyword_counts[keyword] = count
                    score += count * 2  # Более высокий вес для фраз
        
        if score > 0:
            results[category] = {
                'score': score,
                'keywords': keyword_counts.copy()
            }
        else:
            results[category] = {
                'score': 0,
                'keywords': {}
            }
    
    # Затем делаем предобработку с лемматизацией для отдельных слов
    processed_tokens = preprocess_text(text)
    
    # Проверяем совпадения для отдельных слов и леммтизированных словосочетаний
    for category, keywords in categories_dict.items():
        for keyword in keywords:
            # Только для отдельных слов
            if len(keyword.split()) == 1:
                # Лемматизируем ключевое слово
                morph = pymorphy2.MorphAnalyzer()
                lemma = morph.parse(keyword)[0].normal_form
                
                # Проверяем наличие в обработанных токенах
                count = processed_tokens.count(lemma)
                if count > 0:
                    results[category]['keywords'][keyword] = count
                    results[category]['score'] += count
    
    # Расчет относительных весов
    total_score = sum(item['score'] for item in results.values())
    
    if total_score == 0:
        return []
    
    # Формирование отсортированных результатов
    results_list = [(category, data['score'], data['score']/total_score, data['keywords']) 
                    for category, data in results.items()]
    results_list.sort(key=lambda x: x[1], reverse=True)
    
    return results_list[:top_n]

def generate_report(file_path, results, output_format='text'):
    """
    Генерация отчета о классификации.
    
    :param file_path: Путь к классифицируемому файлу
    :param results: Результаты классификации
    :param output_format: Формат вывода (text или json)
    :return: Отчет в указанном формате
    """
    if not results:
        return "Классификация не удалась или не найдено совпадений с категориями."
    
    if output_format == 'json':
        report = {
            "file": os.path.basename(file_path),
            "classification_results": [
                {
                    "category": category,
                    "score": score,
                    "confidence": confidence,
                    "key_terms": list(keywords.keys())
                } for category, score, confidence, keywords in results
            ]
        }
        return json.dumps(report, ensure_ascii=False, indent=2)
    else:
        lines = [f"Результаты классификации для файла: {os.path.basename(file_path)}"]
        lines.append("=" * 80)
        
        for i, (category, score, confidence, keywords) in enumerate(results):
            lines.append(f"{i+1}. {category}")
            lines.append(f"   Уверенность: {confidence:.2%}")
            lines.append(f"   Количество совпадений: {score}")
            
            if keywords:
                lines.append("   Ключевые термины:")
                sorted_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)
                for term, count in sorted_keywords[:5]:  # Показываем топ-5 ключевых термина
                    lines.append(f"     - {term}: {count}")
            
            lines.append("-" * 40)
        
        return "\n".join(lines)

def classify_technical_document(file_path, custom_categories=None):
    """
    Полный процесс классификации технического документа.
    
    :param file_path: Путь к файлу
    :param custom_categories: Пользовательские категории или None для использования предопределенных
    :return: Результаты классификации
    """
    # Чтение текста из файла
    text = read_text_from_file(file_path)
    if not text:
        print(f"Не удалось прочитать файл: {file_path}")
        return None
    
    # Получение категорий
    if custom_categories is None:
        categories = create_technical_categories()
    else:
        categories = custom_categories
    
    # Классификация
    results = classify_with_keywords(text, categories)
    
    # Генерация и вывод отчета
    report = generate_report(file_path, results)
    print(report)
    
    return results

# Пример использования
if __name__ == "__main__":
    file_path = 'Masterpact.pdf'  # Замените на путь к вашему файлу
    results = classify_technical_document(file_path)
    
    # Если вы хотите сохранить отчет в JSON
    # json_report = generate_report(file_path, results, output_format='json')
    # with open('classification_report.json', 'w', encoding='utf-8') as f:
    #     f.write(json_report)

Token indices sequence length is longer than the specified maximum sequence length for this model (44165 > 512). Running this sequence through the model will result in indexing errors


Результаты классификации для файла: Masterpact.pdf
1. Электрические автоматы и выключатели
   Уверенность: 40.00%
   Количество совпадений: 4
   Ключевые термины:
     - автоматический выключатель: 1
     - короткое замыкание: 1
----------------------------------------
2. Техническое обслуживание оборудования
   Уверенность: 40.00%
   Количество совпадений: 4
   Ключевые термины:
     - срок службы: 2
----------------------------------------
3. Электропитание и распределение
   Уверенность: 20.00%
   Количество совпадений: 2
   Ключевые термины:
     - источник питания: 1
----------------------------------------


In [10]:
!python -m spacy download ru_core_news_md

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 2.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [12]:
import re
import spacy
from typing import List, Dict, Any

class TextSegmenter:
    """Класс для сегментации текста на предложения с учетом специальных случаев"""
    
    def __init__(self, language: str = "ru", use_ml: bool = True):
        """
        Инициализирует сегментатор текста
        
        Args:
            language: Код языка (например, 'ru', 'en')
            use_ml: Использовать ли ML-модели для сегментации
        """
        self.language = language
        self.use_ml = use_ml
        
        # Загрузка языковой модели spaCy
        if self.use_ml:
            try:
                if language == "ru":
                    # Try to load smaller Russian model first
                    try:
                        self.nlp = spacy.load("ru_core_news_sm")
                    except OSError:
                        # Fall back to medium model if small is not available
                        self.nlp = spacy.load("ru_core_news_md")
                elif language == "en":
                    # Try to load smaller English model first
                    try:
                        self.nlp = spacy.load("en_core_web_sm")
                    except OSError:
                        # Fall back to medium model if small is not available
                        self.nlp = spacy.load("en_core_web_md")
                else:
                    # Fallback на мультиязычную модель
                    self.nlp = spacy.load("xx_ent_wiki_sm")
            except OSError:
                print(f"Warning: Could not load spaCy model for {language}. Falling back to rule-based approach.")
                self.use_ml = False
        
        # Список распространенных сокращений
        self.abbreviations = self._load_abbreviations(language)
        
        # Регулярные выражения для обработки переносов строк
        self.line_break_pattern = re.compile(r'\s*\n\s*')
        
        # Регулярные выражения для обнаружения конца предложения
        self.sentence_end_pattern = self._create_sentence_end_pattern()
    
    def _load_abbreviations(self, language: str) -> List[str]:
        """Загружает список сокращений для заданного языка"""
        if language == "ru":
            return ["т.е.", "т.д.", "т.п.", "и т.д.", "и т.п.", "др.", "пр.", "см.", "напр.", "проф.", "доц.", "акад."]
        elif language == "en":
            return ["e.g.", "i.e.", "etc.", "vs.", "Mr.", "Mrs.", "Dr.", "Prof.", "Ph.D.", "Inc.", "Ltd."]
        else:
            # Базовый список сокращений
            return ["etc.", "vs."]
    
    def _create_sentence_end_pattern(self) -> re.Pattern:
        """Создает регулярное выражение для определения конца предложения"""
        # Базовый шаблон для обнаружения потенциальных концов предложений
        pattern = r'[.!?]\s+'
        return re.compile(pattern)
    
    def _is_abbreviation_ending(self, text: str, pos: int) -> bool:
        """
        Проверяет, является ли точка в данной позиции частью сокращения
        
        Args:
            text: Текст
            pos: Позиция точки
            
        Returns:
            True, если точка - часть сокращения, иначе False
        """
        # Проверка в обратном порядке от более длинных сокращений к более коротким
        sorted_abbrs = sorted(self.abbreviations, key=len, reverse=True)
        
        for abbr in sorted_abbrs:
            abbr_len = len(abbr)
            if pos >= abbr_len - 1:  # -1 потому что abbr включает точку
                potential_abbr = text[pos-abbr_len+1:pos+1]
                if potential_abbr == abbr:
                    return True
                
        # Проверка на инициалы типа "А."
        if pos > 0 and text[pos] == '.' and text[pos-1].isalpha() and (pos == 1 or not text[pos-2].isalpha()):
            return True
            
        return False
    
    def _preprocess_text(self, text: str) -> str:
        """
        Предварительная обработка текста перед сегментацией
        
        Args:
            text: Исходный текст
            
        Returns:
            Предобработанный текст
        """
        # Нормализация пробелов
        text = re.sub(r'\s+', ' ', text)
        
        # Обработка переносов строк
        text = self._handle_line_breaks(text)
        
        return text
    
    def _handle_line_breaks(self, text: str) -> str:
        """
        Обрабатывает переносы строк, объединяя разорванные предложения
        
        Args:
            text: Исходный текст
            
        Returns:
            Текст с обработанными переносами строк
        """
        # Заменяем одиночные переносы строк на пробелы
        # но сохраняем двойные переносы (они обычно означают новый параграф)
        text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
        
        # Удаляем лишние пробелы
        text = re.sub(r'\s+', ' ', text)
        
        return text
    
    def segment_text_rule_based(self, text: str) -> List[str]:
        """
        Сегментирует текст на предложения с помощью правил
        
        Args:
            text: Исходный текст
            
        Returns:
            Список предложений
        """
        # Предварительная обработка
        text = self._preprocess_text(text)
        
        # Разделение на предложения с учетом сокращений
        segments = []
        start = 0
        
        # Находим все потенциальные концы предложений
        for match in self.sentence_end_pattern.finditer(text):
            end_pos = match.start()
            
            # Проверяем, не является ли это сокращением
            if text[end_pos] == '.' and self._is_abbreviation_ending(text, end_pos):
                continue
            
            # Это конец предложения
            end = end_pos + 1  # Включаем знак препинания
            segments.append(text[start:end].strip())
            start = match.end()
        
        # Добавляем последний сегмент
        if start < len(text):
            segments.append(text[start:].strip())
        
        return segments
    
    def segment_text_ml(self, text: str) -> List[str]:
        """
        Сегментирует текст на предложения с помощью ML
        
        Args:
            text: Исходный текст
            
        Returns:
            Список предложений
        """
        # Предварительная обработка
        text = self._preprocess_text(text)
        
        # Использование spaCy для сегментации
        doc = self.nlp(text)
        segments = [sent.text.strip() for sent in doc.sents]
        
        return segments
    
    def segment_text(self, text: str) -> List[str]:
        """
        Сегментирует текст на предложения
        
        Args:
            text: Исходный текст
            
        Returns:
            Список предложений
        """
        if self.use_ml:
            return self.segment_text_ml(text)
        else:
            return self.segment_text_rule_based(text)


# Пример использования
if __name__ == "__main__":
    # Тестовый текст
    test_text = """Со мной случилась такая 
вот оказия, т.е. происшествие: сегодня я гулял по 
Садовой улице и наткнулся на выхухоль."""
    
    # Создание сегментатора с автоматическим fallback на rule-based подход
    segmenter = TextSegmenter(language="ru", use_ml=True)
    
    # Сегментация текста
    segments = segmenter.segment_text(test_text)
    
    # Вывод результатов
    print("Исходный текст:")
    print(test_text)
    print("\nСегментированный текст:")
    for i, segment in enumerate(segments):
        print(f"{i+1}. {segment}")

Исходный текст:
Со мной случилась такая 
вот оказия, т.е. происшествие: сегодня я гулял по 
Садовой улице и наткнулся на выхухоль.

Сегментированный текст:
1. Со мной случилась такая вот оказия, т.е. происшествие: сегодня я гулял по Садовой улице и наткнулся на выхухоль.
